In [32]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_circles
from sklearn.neural_network import MLPClassifier

from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.metrics import accuracy_score
from sklearn.inspection import DecisionBoundaryDisplay

from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [29]:
class l_svm():
    
    def __init__(self, lr=0.01, itr=1000):
        self.lr = lr
        self.itr = itr
        self.w = None
        self.b = None
        
    def fit(self, X,y):
        samples, feat = X.shape
        
        self.w = np.zeros(feat)
        self.b = 0
        
        labels = np.ones(samples)
        
        j = 0
        for s in y:
            if s <= 0:
                labels[j] = -1
            j += 1
            
        for i in range(self.itr):
            c = 0
            for x_i in X: 
                # check if y * (wx - b) >=1 and udpate weights accordingly
                if (labels[c] * (np.dot(x_i, self.w) - self.b)) >= 1:
                    self.w = self.w - self.lr * self.w
                else:
                    self.w = self.w - self.lr * (self.w - np.dot(x_i, labels[c]))
                    self.b = self.b - self.lr * labels[c]
                c += 1
                
        
    def predict(self, X):
        pred = np.dot(X, self.w) - self.b   
        return pred
    
    def get_acc(self, X, y_actual):
       
        pred = np.sign(self.predict(X)) 
        count = 0
        for i in range(len(pred)):
            if pred[i] == y_test[i]:
                count += 1
        
        return count/len(pred)
    
    
    def plot_db(self, X, y):
        
        plt.scatter(X[:,0], X[:,1], c=y)
        ax = plt.gca()
        x_b = ax.get_xlim()
        y_b = ax.get_ylim()
        xx = np.linspace(x_b[0], x_b[1], 50)
        yy = np.linspace(y_b[0], y_b[1], 50)
        YY, XX = np.meshgrid(yy, xx)
        xy = np.vstack([XX.ravel(), YY.ravel()]).T
        Z = self.predict(xy).reshape(XX.shape)
        ax.contour(XX, YY, Z, levels=[-1, 0, 1],linestyles=['--', '-', '--'])
        
        plt.show()
        
        return
        


In [24]:
class log_reg():
    def __init__(self, lr=0.01, itr=1000):
        self.lr = lr
        self.itr = itr
        self.w = None
        self.b = None
        
        
    def sig(self, x):
        return 1.0/(1+np.exp(-x))

    def calc_gradient(self, X, y, y_hat, m):
        grad = (1/m)*np.dot(X.T,(y_hat - y))   
        return grad
    
    def fit(self, X, y):
        m,n = X.shape
        self.w = np.zeros(n)
    
        for i in range(self.itr):
            y_hat = self.sig(np.dot(X, self.w))
            grad = self.calc_gradient(X, y, y_hat, m)
            self.w = self.w - self.lr * grad
            
        return 
        
      
    def predict(self, X):
        return self.sig(np.dot(X, self.w))
        
        labels = []
              
        for pred in predictions:
            if pred >= 0.5:
                labels.append(1)
            else:
                labels.append(-1)
            
        labels = np.asarray(labels)
        return labels
        
    
    def get_acc(self, X, y):
        predictions = self.predict(X)
        
        labels = []
       
        for pred in predictions:
            if pred >= 0.5:
                labels.append(1)
            else:
                labels.append(-1)
            
        labels = np.asarray(labels)
        
        count = 0
        for i in range(len(labels)):
            if labels[i] == y[i]:
                count += 1
        
        return count/len(labels)
        
    def plot_db(self, X, y):
        
        plt.scatter(X[:,0], X[:,1], c=y)
        x_values = [np.min(X[:, 0]), np.max(X[:, 1])]
        y_values = - (np.dot(self.w[0], x_values)) / self.w[1]

        plt.plot(x_values, y_values, label='Decision Boundary')
        
        plt.show()
        return

In [33]:
class kernel_svm():
    def __init__(self, lr=0.01, itr=1000, kernel='polynomial', p=2, g=0.1, C=1):
        self.lr = lr
        self.itr = itr
        self.kernel = kernel
        self.p = p
        self.g = g
        self.C = C
        self.y_t = None
        self.X_t = None
        self.w = None
        self.b = None
    
    
    
    def calc_kernel(self, X, Y):
        if self.kernel == 'polynomial':
            new_X = (np.dot(X, Y.T) + self.C)**self.p
            return new_X
        elif self.kernel == 'rbf':
            new_X = np.exp(-self.g * np.linalg.norm(X[:, np.newaxis] - Y[np.newaxis, :], axis=2) ** 2)
            return new_X
            
        return X
    
    def fit(self, X,y):
        self.X_t = X
        samples, feat = X.shape
        
        self.w = np.random.random(samples)
        self.b = 0
        
        labels = np.ones(samples)
        
        j = 0
        for s in y:
            if s <= 0:
                labels[j] = -1
            j += 1
            
        self.y_t = labels
        
        new_X = self.calc_kernel(X, X)

        for i in range(self.itr):
            grad = np.ones(samples) - np.dot(np.outer(labels,labels)*new_X, self.w)
            
            self.w = self.w + self.lr * grad
            
            #constraints
            for j in range(len(self.w)):
                if(self.w[j] > self.C):
                    self.w[j] = self.C
                elif(self.w[j] < 0):
                    self.w[j] = 0
            
        
        b_s = []
        for a in range(len(self.w)):
            if (self.w[a] < self.C and self.w[a] > 0):
                b_s.append(labels[a] - (self.w *labels).dot(self.calc_kernel(X, X[a])))
           
        return
        
     
    
    def get_acc(self, X, y_actual):
        X = self.calc_kernel(self.X_t, X)
        
        pred = np.dot(self.w * self.y_t, X) - self.b   
        pred = np.sign(pred)
        
        count = 0
        for i in range(len(pred)):
            if pred[i] == y_test[i]:
                count += 1
        
        return count/len(pred)
    
    def predict(self, X):
        X = self.calc_kernel(self.X_t, X)
        pred = np.dot(self.w * self.y_t, X) - self.b   
 
        return pred
    
    def plot_db(self, X, y):
        plt.scatter(X[:,0], X[:,1], c=y)
        ax = plt.gca()
        x_b = ax.get_xlim()
        y_b = ax.get_ylim()
        xx = np.linspace(x_b[0], x_b[1], 50)
        yy = np.linspace(y_b[0], y_b[1], 50)
        YY, XX = np.meshgrid(yy, xx)
        xy = np.vstack([XX.ravel(), YY.ravel()]).T
        Z = self.predict(xy).reshape(XX.shape)
        ax.contour(XX, YY, Z, levels=[-1, 0, 1],linestyles=['--', '-', '--'])
       
        plt.show()

In [34]:
class kernel_log_reg():
    def __init__(self, lr=0.01, itr=1000):
        self.lr = lr
        self.itr = itr
        self.w = None
        self.X = None
        self.y = None
        self.a = None
        
     
    
    def kernel(self, X, y):
        k = np.zeros([X.shape[0], y.shape[0]])
        
        for i in range(X.shape[0]):
            for j in range(y.shape[0]):
                if ((np.rint(X[i]) == np.rint(y[j])).all()):
                    k[i][j] = 1
                else:
                    k[i][j] = 0
                    
        return k
     
    
    def sig(self, x):
        return 1.0/(1+np.exp(-x))

    def calc_gradient(self, X, y):
        grad = np.dot(X, y - self.sig(np.dot(X, self.a)))   
        return grad
    
    def fit(self, X, y):
        m,n = X.shape
        self.X = X
        self.y = y
        
        self.a = np.zeros(m)
        K = self.kernel(X,X)
        for i in range(self.itr):
            grad = self.calc_gradient(K, y)
            self.a = self.a + grad * self.lr
        return 
        
    def predict(self, X):
        predictions = self.sig(np.dot((self.kernel(self.X, X)).T, self.a))
        #print(predictions)
        #print(self.a)
        labels = []
              
        for pred in predictions:
            if pred >= 0.5:
                labels.append(1)
            else:
                labels.append(-1)
            
        labels = np.asarray(labels)
        return labels
    
    def get_acc(self, X, y_test):
        
        pred = self.predict(X)
        count = 0
        for i in range(len(pred)):
            if pred[i] == y_test[i]:
                count += 1
        
        return count/len(pred)

    def plot_db(self, X, y):
        plt.scatter(X[:,0], X[:,1], c=y)
        ax = plt.gca()
        x_b = ax.get_xlim()
        y_b = ax.get_ylim()
        xx = np.linspace(x_b[0], x_b[1], 50)
        yy = np.linspace(y_b[0], y_b[1], 50)
        YY, XX = np.meshgrid(yy, xx)
        xy = np.vstack([XX.ravel(), YY.ravel()]).T
        Z = self.predict(xy).reshape(XX.shape)
        ax.contour(XX, YY, Z, levels=[-1, 0, 1],linestyles=['--', '-', '--'])
        
        plt.show()
        
        
    

In [14]:
def besthyperparms_knn(x, y):
  # Finding the best set of hyperparameters for knn using sklearn gridsearch CV
  #Consider three hyperparameters# 1. 
  #n_neighbors: Decide the best k 
  #weights: adding the weights to data points. 'uniform' ->no weight, 
  #'distance' weighs points by the inverse of their distances meaning nearer points will have more weight than the farther points.
  #metric: The distance metric for similarity.
  grid_params = { 'n_neighbors' : [5,7,9,11,13,15],
                'weights' : ['uniform','distance'],
                'metric' : ['minkowski','euclidean','manhattan']}
  gs = GridSearchCV(neighbors.KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
  g_res = gs.fit(x, y)
  print ("Best Score: ",g_res.best_score_)
  print ("Best Hyperparameters: ", g_res.best_params_)

In [15]:
from sklearn import datasets
data = datasets.load_breast_cancer()
print(data.keys())
#print(data.DESCR)
# Store the feature data
X = data.data
# store the target data
y = data.target
samp, feat = X.shape
print(X.shape, y.shape)
print ("Features:%d, samples:%d"%(feat, samp))
# split the data using Scikit-Learn's train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
(569, 30) (569,)
Features:30, samples:569


In [16]:
#get the best hyperparameters for knn for this dataset
besthyperparms_knn(X_train, y_train)

Fitting 3 folds for each of 36 candidates, totalling 108 fits
Best Score:  0.931924882629108
Best Hyperparameters:  {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}


# Find accuracy for each classifier

In [41]:
knn = neighbors.KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='manhattan')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
knn_accuracy = accuracy_score(y_test, y_pred)

In [42]:
nn = MLPClassifier(alpha=1, hidden_layer_sizes=(30,), 
                     random_state=1,max_iter=1000)
nn.fit(X_train, y_train)
y_pred = nn.predict(X_test)
nn_accuracy = accuracy_score(y_test, y_pred)

In [43]:
lsvm = l_svm()
lsvm.fit(X_train, y_train)

pred = np.sign(lsvm.predict(X_test))

count = 0
for i in range(len(pred)):
  if pred[i] <= 0 and y_test[i] == 0:
    count += 1
  elif pred[i] > 0 and y_test[i] > 0:
    count += 1

lsvm_acc = count/len(pred)

In [44]:
lg = log_reg()
lg.fit(X_train, y_train)
lg_acc = lg.get_acc(X_test, y_test)

In [45]:
k_lg = kernel_log_reg()
k_lg.fit(X_train, y_train)
k_lg_acc = k_lg.get_acc(X_test, y_test)

In [46]:
ksvm_rbf = kernel_svm(kernel='rbf', g=1.2, C=10)
ksvm_rbf.fit(X_train, y_train)
#print("Accuracy polynomial SVM", ksvm_poly.get_acc(X_test, y_test))

pred = np.sign(ksvm_rbf.predict(X_test))

count = 0
for i in range(len(pred)):
  if pred[i] <= 0 and y_test[i] == 0:
    count += 1
  elif pred[i] > 0 and y_test[i] > 0:
    count += 1

rbf_acc = count/len(pred)

ksvm_poly = kernel_svm(kernel='polynomial', p=4, C=2)
ksvm_poly.fit(X_train, y_train)
#print("Accuracy polynomial SVM", ksvm_poly.get_acc(X_test, y_test))

pred = np.sign(ksvm_poly.predict(X_test))

count = 0
for i in range(len(pred)):
  if pred[i] <= 0 and y_test[i] == 0:
    count += 1
  elif pred[i] > 0 and y_test[i] > 0:
    count += 1

poly_acc = count/len(pred)

# Results on Cancer Dataset

In [48]:
print("Accuracy KNN:", knn_accuracy)
print("Accuracy NN:", nn_accuracy)
print("Accuracy Linear SVM: ", lsvm_acc)
print("Accuracy Log Reg: ", lg_acc)
print("Accuracy Kernel Log Reg: ", k_lg_acc)
print("Accuracy RBF SVM: ", rbf_acc)
print("Accuracy Polynomial SVM: ", poly_acc)

Accuracy KNN: 0.9440559440559441
Accuracy NN: 0.9230769230769231
Accuracy Linear SVM:  0.4405594405594406
Accuracy Log Reg:  0.48951048951048953
Accuracy Kernel Log Reg:  0.5594405594405595
Accuracy RBF SVM:  0.9370629370629371
Accuracy Polynomial SVM:  0.4405594405594406


# L1 regularization

In [79]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.datasets import load_breast_cancer

print(X_train.shape)
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False,max_iter=1000).fit(X_train, y_train) 
model = SelectFromModel(lsvc, prefit=True) 
X_train_n = model.transform(X_train) 
print(X_train_n.shape)
features = model.get_support()

data = load_breast_cancer()

for i in range(30):
    if features[i] == True:
        print(data.feature_names[i])

(426, 30)
(426, 5)
mean perimeter
mean area
area error
worst texture
worst area
